### Bilinear capacity -- numerical (multiple S/P)

In [1]:
import sys
sys.path.insert(0, '../../../network')

In [2]:
import time
import itertools
from tqdm import tqdm
import ray
import numpy as np

In [3]:
from network import Population, RateNetwork
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse
from sequences import GaussianSequence

In [4]:
ray.init(redis_address="10.122.160.26:6382", include_webui=True, ignore_reinit_error=True)

{'node_ip_address': '10.122.160.26',
 'redis_address': '10.122.160.26:6382',
 'object_store_address': '/tmp/ray/session_2020-03-13_23-32-59_134083_2290970/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-13_23-32-59_134083_2290970/sockets/raylet',
 'webui_url': 'http://10.122.160.26:8080/?token=a0a17cdfca7fa34e5ac0aaa9a39766016cd09678855e94ba',
 'session_dir': '/tmp/ray/session_2020-03-13_23-32-59_134083_2290970'}

In [5]:
@ray.remote
def f():
    time.sleep(0.01)
    return ray.services.get_node_ip_address()

# Get a list of the IP addresses of the nodes that have joined the cluster.
print(set(ray.get([f.remote() for _ in range(1000)])))

{'10.122.160.26'}


In [6]:
def run_simulation(S, P, N=50000):
    phi = ErrorFunction(mu=0.22, sigma=0.1)
    exc = Population(N, tau=1e-2, phi=phi.phi)
    c = np.sqrt(exc.size)/exc.size
    sequences = [GaussianSequence(P,exc.size,seed=i) for i in range(S)] 
    patterns = np.stack([s.inputs for s in sequences])
    conn = SparseConnectivity(source=exc, target=exc, p=c, disable_pbar=True)
    synapse = LinearSynapse(conn.K, A=1)
    conn.store_sequences(patterns, synapse.h_EE)
    net = RateNetwork(
        exc,
        c_EE=conn,
        formulation=1,
        disable_pbar=True) 
    net.simulate(
        t=0.1+0.01*P,
        r0=exc.phi(patterns[0,0,:]))
    overlaps = sequences[0].overlaps(
        net,
        exc,
        correlation=False,
        disable_pbar=True)
    correlations = sequences[0].overlaps(
        net,
        exc,
        correlation=True,
        disable_pbar=True)
    return {
        'params': {
            'theta': phi.mu,
            'sigma': phi.sigma,
            'N': N,
            'c': c,
            'S': S,
            'P': P,
        },
        'overlaps': overlaps,
        'correlations': correlations,
    }

In [7]:
N = 100000 
S = [1, 2, 4, 8, 16] 
P = [8, 16, 32, 64, 128]

combinations = list(itertools.product(
    np.atleast_1d(N),
    np.atleast_1d(S),
    np.atleast_1d(P)))

parallel = True
object_ids = []
run_simulation_ray = ray.remote(num_cpus=10)(run_simulation)

n = 0
for N, S, P in combinations[:]:
    if S*P > np.sqrt(N):
        continue
    if parallel:
        func = run_simulation_ray.remote
    else:
        func = run_simulation
    object_ids.append(func(S,P,N))
    n += 1

In [ ]:
directory = "data/"
pbar = tqdm(total=n)
while len(object_ids) > 0:
    if parallel:
        ready_object_ids, _ = ray.wait(object_ids)
        id_ = ready_object_ids[0]
        data = ray.get(id_)
        object_ids.remove(id_)
    else:
        data = object_ids[0]
        object_ids.remove(data)
    params = data['params']
    N, S, P = params['N'], params['S'], params['P']
    filename = "N%i_S%i_P%i"%(N,S,P) + ".npy"
    filepath = directory + filename
    np.save(open(filepath, 'wb'), data)
    pbar.update(1)
    time.sleep(1)

  0%|          | 0/19 [00:00<?, ?it/s]